импорты и пути

In [1]:
import base64, json, zlib, cv2, numpy as np, tqdm
from pathlib import Path

ROOT = Path.cwd().parent
CITY_DIR      = ROOT / 'data' / 'cityscapes'
POT_JSON_DIR  = ROOT / 'data' / 'potholes' / 'masks'
POT_IMG_DIR   = ROOT / 'data' / 'potholes' / 'images'
POT_MASK_DIR  = ROOT / 'data' / 'potholes' / 'masks_png'
POT_MASK_DIR.mkdir(parents=True, exist_ok=True)

CITY_MAP    = {7: 1, 8: 2}                       # Cityscapes → наши id
POT_CLS_MAP = {"road": 1, "crack": 1, "pothole": 3}  # JSON → наши id

print("ROOT →", ROOT)
print("Cityscapes exists:", CITY_DIR.exists())
print("Pothole JSON dir  :", POT_JSON_DIR.exists())

ROOT → /Users/macbook/projects/road_seg
Cityscapes exists: True
Pothole JSON dir  : True


Helper: decode Supervisely bitmap

In [2]:
def _decode_bitmap(bmp_dict):
    """
    Декодирует Supervisely-bitmap → 2-D бинарная маска + (ox, oy).

    В JSON хранится base64(zlib(png)).
    PNG бывает 1-канальный (градации серого) или 4-канальный (RGBA).
    Всегда возвращаем ndarray uint8 формы (H, W) со значениями 0/1.
    """
    import base64, zlib, cv2, numpy as np

    comp = base64.b64decode(bmp_dict["data"])
    png_bytes = zlib.decompress(comp)
    img = cv2.imdecode(np.frombuffer(png_bytes, np.uint8), cv2.IMREAD_UNCHANGED)
    if img is None:
        raise ValueError("Bitmap decode failed")

    # приводим к 2-D: если RGBA, берём альфа-канал
    if img.ndim == 3:
        mask = (img[:, :, -1] > 0).astype(np.uint8)
    else:
        mask = (img > 0).astype(np.uint8)

    ox, oy = bmp_dict["origin"]
    return mask, (ox, oy)


def _draw_polygon(poly_points, canvas, cls_id):
    """Закрашивает полигоны (список точек [[x,y],...]) на canvas указанным cls_id."""
    pts = np.round(np.array(poly_points, dtype=np.float32)).astype(np.int32)
    cv2.fillPoly(canvas, [pts], int(cls_id))

конвертация JSON → PNG

In [3]:
def convert_pothole_json_to_png():
    json_files = list(POT_JSON_DIR.glob('*.json'))
    if not json_files:
        print("❗ JSON‑файлы не найдены, пропускаю конвертацию.")
        return

    for js in tqdm.tqdm(json_files, desc='json→png'):
        with open(js) as f:
            ann = json.load(f)
        H, W = ann["size"]["height"], ann["size"]["width"]
        full = np.zeros((H, W), np.uint8)

        for obj in ann["objects"]:
            cls_id = POT_CLS_MAP.get(obj.get("classTitle", ""), 0)
            if cls_id == 0:
                continue

            if "bitmap" in obj:  # быстрый путь
                m, (ox, oy) = _decode_bitmap(obj["bitmap"])
                full[oy:oy+m.shape[0], ox:ox+m.shape[1]][m > 0] = cls_id

            elif "points" in obj:  # polygon/point
                _draw_polygon(obj["points"]["exterior"], full, cls_id)

        out_path = POT_MASK_DIR / (js.stem.split('.jpg')[0] + '.png')
        cv2.imwrite(str(out_path), full)

функции ремапа

In [4]:
def remap_cityscapes():
    masks = list(CITY_DIR.rglob('*_labelIds.png'))
    if not masks:
        print("❗ Cityscapes *_labelIds.png не найдены – проверьте распаковку.")
        return
    for p in tqdm.tqdm(masks, desc='city‑remap'):
        m = cv2.imread(str(p), cv2.IMREAD_UNCHANGED)
        out = np.zeros_like(m, np.uint8)
        for src, dst in CITY_MAP.items():
            out[m == src] = dst
        cv2.imwrite(str(p).replace('_labelIds', '_ourIds'), out)

In [5]:
def remap_potholes():
    for p in tqdm.tqdm(POT_MASK_DIR.glob('*.png'), desc='pothole‑remap'):
        m = cv2.imread(str(p), cv2.IMREAD_UNCHANGED)
        if m is None:
            continue
        m[m == 3] = 3
        m[(m == 1) | (m == 2)] = 1
        cv2.imwrite(str(p), m)

Проверка

In [6]:
convert_pothole_json_to_png()
remap_potholes()
remap_cityscapes()

stats = {
    'city imgs'    : len(list((CITY_DIR).rglob('*.png'))),
    'city masks'   : len(list((CITY_DIR).rglob('*_ourIds.png'))),
    'pothole imgs' : len(list(POT_IMG_DIR.glob('*.jpg'))),
    'pothole masks': len(list(POT_MASK_DIR.glob('*.png')))
}
print("\nDone ✔ Stats:")
for k, v in stats.items():
    print(f"{k:14}: {v}")

json→png: 100%|██████████| 2235/2235 [00:29<00:00, 76.61it/s]
pothole‑remap: 2235it [00:13, 165.94it/s]
city‑remap: 100%|██████████| 5000/5000 [01:17<00:00, 64.28it/s]



Done ✔ Stats:
city imgs     : 25000
city masks    : 5000
pothole imgs  : 2235
pothole masks : 2235
